# Claude Skills for Financial Applications

Build real-world financial dashboards, portfolio analytics, and automated reporting workflows using Claude's Excel, PowerPoint, and PDF skills.

**What you'll learn:**
- Create comprehensive financial models in Excel with formulas and charts
- Generate executive presentations from financial data
- Build portfolio analysis tools with risk metrics
- Automate multi-format reporting pipelines
- Optimize token usage for production workflows

## Table of Contents

1. [Setup & Data Loading](#setup)
2. [Use Case 1: Financial Dashboard Creation](#financial-dashboard)
   - [Excel Financial Model](#excel-model)
   - [Executive PowerPoint](#executive-ppt)
   - [PDF Financial Report](#pdf-report)
3. [Use Case 2: Portfolio Analysis Workflow](#portfolio-analysis)
   - [Portfolio Analytics Excel](#portfolio-excel)
   - [Investment Committee Deck](#investment-deck)
4. [Use Case 3: Automated Reporting Pipeline](#reporting-pipeline)
5. [Best Practices & Production Tips](#best-practices)

## Prerequisites

This notebook assumes you've completed **Notebook 1: Introduction to Skills**.

If you haven't:
1. Complete the setup in Notebook 1 first
2. Verify your environment with the test cells
3. Ensure you can create and download files

**Required:**
- Anthropic API key configured
- SDK version 0.69.0 installed from whl
- Virtual environment activated

## 1. Setup & Data Loading {#setup}

Let's start by importing our dependencies and loading the financial data we'll work with throughout this notebook.

In [ ]:
# Standard imports
import os
import sys
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta

# Add parent directory for imports
sys.path.insert(0, str(Path.cwd().parent))

# Anthropic SDK
from anthropic import Anthropic
from dotenv import load_dotenv

# Our utilities
from file_utils import (
    extract_file_ids,
    download_all_files,
    get_file_info,
    print_download_summary
)

# Load environment
load_dotenv(Path.cwd().parent / '.env')

# Configuration
API_KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = "claude-sonnet-4-5-20250929"

if not API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not found. Please configure your .env file.")

# Initialize client
client = Anthropic(api_key=API_KEY)

# Setup directories
OUTPUT_DIR = Path.cwd().parent / "outputs" / "financial"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = Path.cwd().parent / "sample_data"

print("✓ Environment configured")
print(f"✓ Output directory: {OUTPUT_DIR}")
print(f"✓ Data directory: {DATA_DIR}")

### Load Financial Data

We have four datasets representing different aspects of a company's financial position:

In [ ]:
# Load financial statements
financial_statements = pd.read_csv(DATA_DIR / "financial_statements.csv")
print("📊 Financial Statements Overview:")
print(f"   Shape: {financial_statements.shape}")
print(f"   Categories: {len(financial_statements['Category'].unique())} financial metrics")
print(f"   Quarters: {list(financial_statements.columns[1:5])}")
print()

# Show sample data
print("Sample data (first 5 rows):")
financial_statements.head()

In [ ]:
# Load portfolio holdings
with open(DATA_DIR / "portfolio_holdings.json", "r") as f:
    portfolio_data = json.load(f)

print("💼 Portfolio Overview:")
print(f"   Portfolio: {portfolio_data['portfolio_name']}")
print(f"   Total Value: ${portfolio_data['total_value']:,.2f}")
print(f"   Holdings: {len(portfolio_data['holdings'])} stocks")
print(f"   Cash Position: ${portfolio_data['cash_position']['amount']:,.2f}")
print(f"   Total Return: {portfolio_data['performance_metrics']['total_return_percent']:.1f}%")
print()

# Convert holdings to DataFrame for easier manipulation
portfolio_df = pd.DataFrame(portfolio_data['holdings'])
print("Top 5 holdings by value:")
portfolio_df.nlargest(5, 'market_value')[['ticker', 'name', 'market_value', 'unrealized_gain']]

In [ ]:
# Load quarterly metrics
with open(DATA_DIR / "quarterly_metrics.json", "r") as f:
    quarterly_metrics = json.load(f)

print("📈 Quarterly Metrics Overview:")
print(f"   Quarters available: {len(quarterly_metrics['quarters'])}")
print(f"   Metrics per quarter: {len(quarterly_metrics['quarters'][0])} KPIs")
print()

# Show latest quarter metrics
latest_quarter = quarterly_metrics['quarters'][-1]
print(f"Latest Quarter ({latest_quarter['quarter']}):")
for key, value in latest_quarter.items():
    if key != 'quarter' and isinstance(value, (int, float)):
        if 'revenue' in key.lower() or 'cost' in key.lower():
            print(f"   {key.replace('_', ' ').title()}: ${value:,.0f}")
        elif 'percent' in key.lower() or 'margin' in key.lower() or 'rate' in key.lower():
            print(f"   {key.replace('_', ' ').title()}: {value:.1f}%")
        else:
            print(f"   {key.replace('_', ' ').title()}: {value:,.0f}")

### Helper Functions

Let's define some helper functions for this notebook:

In [ ]:
def create_skills_message(client, prompt, skills, prefix="", show_token_usage=True):
    """
    Helper function to create messages with Skills.
    
    Args:
        client: Anthropic client
        prompt: User prompt
        skills: List of skill dicts [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}]
        prefix: Prefix for downloaded files
        show_token_usage: Whether to print token usage
    
    Returns:
        Tuple of (response, download_results)
    """
    response = client.beta.messages.create(
        model=MODEL,
        max_tokens=4096,
        container={"skills": skills},
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[{"role": "user", "content": prompt}],
        betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"]
    )
    
    if show_token_usage:
        print(f"\n📊 Token Usage: {response.usage.input_tokens} in, {response.usage.output_tokens} out")
    
    # Download files
    results = download_all_files(
        client,
        response,
        output_dir=str(OUTPUT_DIR),
        prefix=prefix
    )
    
    return response, results

def format_financial_value(value, is_currency=True, decimals=0):
    """Format financial values for display."""
    if is_currency:
        return f"${value:,.{decimals}f}"
    else:
        return f"{value:,.{decimals}f}"

print("✓ Helper functions defined")

## 2. Use Case 1: Financial Dashboard Creation {#financial-dashboard}

Let's create a comprehensive financial dashboard that analyzes quarterly performance, generates executive presentations, and produces formal reports.

### 2.1 Excel Financial Model {#excel-model}

We'll create a multi-sheet Excel workbook with:
- P&L Statement with YoY comparison
- Balance Sheet with key ratios
- Cash Flow analysis
- KPI Dashboard with charts

**⏱️ Expected time:** 1-2 minutes

In [ ]:
# Prepare the financial data for the prompt
fs_data = financial_statements.to_dict('records')
quarters_2024 = ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q4_2024']
quarters_2023 = ['Q1_2023', 'Q2_2023', 'Q3_2023', 'Q4_2023']

print("Creating comprehensive financial dashboard Excel file...")
print("This will include P&L, Balance Sheet, Cash Flow, and KPI Dashboard.")
print("\n⏱️ Generation time: 1-2 minutes\n")

financial_dashboard_prompt = f"""
Create a comprehensive financial dashboard Excel workbook with the following sheets:

Sheet 1 - "P&L Statement":
Create a Profit & Loss statement for 2024 quarters with YoY comparison.

Use this data:
{json.dumps({row['Category']: {q: row[q] for q in quarters_2024} for row in fs_data[:14]}, indent=2)}

Include:
- Quarterly columns for Q1-Q4 2024
- YoY growth percentages vs 2023
- Profit margin calculations
- Conditional formatting (green for positive growth, red for negative)

Sheet 2 - "Balance Sheet":
Create a balance sheet with these items from the data:
- Current Assets, Total Assets
- Current Liabilities, Total Liabilities  
- Shareholders Equity
- Calculate: Current Ratio, Debt-to-Equity, ROE

Sheet 3 - "Cash Flow":
Show Operating Cash Flow trends with:
- Quarterly data for 2024
- Free Cash Flow calculation (OCF - CapEx, assume CapEx = 500000/quarter)
- Cash conversion ratio (OCF / Net Income)

Sheet 4 - "KPI Dashboard":
Create a dashboard with:
- Revenue trend chart (line chart)
- Profit margin trend chart (line chart)
- Key metrics summary box with Q4 2024 vs Q4 2023
- YoY Revenue Growth: {((14500000 - 12300000) / 12300000 * 100):.1f}%
- Operating Margin improvement

Apply professional formatting:
- Company colors: headers in dark blue (#003366)
- Currency formatting for all monetary values
- Percentage formatting where appropriate
- Clear section headers and borders
"""

# Create the Excel financial dashboard
excel_response, excel_results = create_skills_message(
    client,
    financial_dashboard_prompt,
    [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}],
    prefix="financial_dashboard_"
)

print("\n" + "="*60)
print_download_summary(excel_results)
print("\n✅ Financial dashboard Excel created successfully!")

### 2.2 Executive PowerPoint Presentation {#executive-ppt}

Now let's create an executive presentation that summarizes the key findings from our financial analysis.

**⏱️ Expected time:** 1-2 minutes

In [ ]:
print("Creating executive presentation from financial metrics...")
print("\n⏱️ Generation time: 1-2 minutes\n")

# Calculate some key metrics for the presentation
q4_2024_revenue = 14500000
q4_2023_revenue = 12300000
yoy_growth = (q4_2024_revenue - q4_2023_revenue) / q4_2023_revenue * 100

q4_2024_net_income = 1878750
q4_2023_net_income = 1209000
net_income_growth = (q4_2024_net_income - q4_2023_net_income) / q4_2023_net_income * 100

executive_ppt_prompt = f"""
Create a 4-slide executive presentation for Q4 2024 financial results:

Slide 1 - Title:
- Title: "Q4 2024 Financial Results"
- Subtitle: "Executive Summary - Acme Corporation"
- Date: January 2025

Slide 2 - Financial Highlights:
- Title: "Q4 2024 Performance Highlights"
- Create a two-column layout:
  Left side - Key Metrics:
  • Revenue: $14.5M (+{yoy_growth:.1f}% YoY)
  • Net Income: $1.88M (+{net_income_growth:.1f}% YoY) 
  • Operating Margin: 17.9% (up 2.9pp)
  • Operating Cash Flow: $2.85M
  
  Right side - Column chart showing quarterly revenue:
  Q1 2024: $12.5M
  Q2 2024: $13.2M
  Q3 2024: $13.8M
  Q4 2024: $14.5M

Slide 3 - Profitability Trends:
- Title: "Margin Expansion & Profitability"
- Add a line chart showing net margin % by quarter:
  Q1 2024: 11.4%
  Q2 2024: 11.8%
  Q3 2024: 12.4%
  Q4 2024: 13.0%
- Add bullet points below:
  • Consistent margin expansion throughout 2024
  • Operating leverage driving profitability
  • Cost optimization initiatives delivering results

Slide 4 - Key Takeaways:
- Title: "Key Takeaways & Outlook"
- Bullet points:
  ✓ Record Q4 revenue of $14.5M
  ✓ 17.9% YoY revenue growth
  ✓ 55% increase in net income YoY
  ✓ Strong cash generation: $2.85M operating cash flow
  ✓ Well-positioned for continued growth in 2025

Use professional corporate design:
- Dark blue (#003366) for headers
- Clean, modern layout
- Data-driven visualizations
"""

# Create the executive presentation
ppt_response, ppt_results = create_skills_message(
    client,
    executive_ppt_prompt,
    [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}],
    prefix="executive_summary_"
)

print("\n" + "="*60)
print_download_summary(ppt_results)
print("\n✅ Executive presentation created successfully!")

### 2.3 PDF Financial Report {#pdf-report}

Let's generate a formal PDF financial report suitable for distribution to stakeholders.

**⏱️ Expected time:** 1-2 minutes

In [ ]:
print("Generating formal PDF financial report...")
print("\n⏱️ Generation time: 1-2 minutes\n")

pdf_report_prompt = f"""
Create a formal PDF financial report for Q4 2024:

ACME CORPORATION
Quarterly Financial Report
Q4 2024
For the Period Ending December 31, 2024

---

EXECUTIVE SUMMARY

Acme Corporation delivered exceptional Q4 2024 results with revenue of $14.5 million,
representing 17.9% year-over-year growth. Net income increased 55.4% to $1.88 million,
demonstrating strong operational efficiency and margin expansion.

KEY FINANCIAL HIGHLIGHTS

Financial Performance (in millions):
                    Q4 2024     Q4 2023     Change
Revenue             $14.50      $12.30      +17.9%
Gross Profit        $5.80       $4.67       +24.2%
Operating Income    $2.60       $1.72       +51.2%
Net Income          $1.88       $1.21       +55.4%

Key Metrics:
• Gross Margin: 40.0% (vs. 38.0% in Q4 2023)
• Operating Margin: 17.9% (vs. 14.0% in Q4 2023)
• Net Margin: 13.0% (vs. 9.8% in Q4 2023)

BALANCE SHEET HIGHLIGHTS

As of December 31, 2024:
• Total Assets: $50.0 million
• Total Liabilities: $15.6 million
• Shareholders' Equity: $34.4 million
• Current Ratio: 3.37x
• Debt-to-Equity: 0.45x

CASH FLOW SUMMARY

Q4 2024 Operating Cash Flow: $2.85 million
• Cash conversion ratio: 152%
• Free Cash Flow: $2.35 million
• Year-to-date OCF: $10.05 million

OUTLOOK

The company maintains a positive outlook for 2025, with continued focus on:
• Revenue growth through market expansion
• Operational efficiency improvements
• Strategic investments in R&D
• Maintaining strong cash generation

---
This report contains forward-looking statements subject to risks and uncertainties.
For more information, contact: investor.relations@acmecorp.com

Format this as a professional financial report PDF with:
- Clear section headers
- Professional typography
- Tables for numerical data
- Corporate formatting
"""

# Create the PDF report
pdf_response, pdf_results = create_skills_message(
    client,
    pdf_report_prompt,
    [{"type": "anthropic", "skill_id": "pdf", "version": "latest"}],
    prefix="q4_2024_report_"
)

print("\n" + "="*60)
print_download_summary(pdf_results)
print("\n✅ PDF financial report created successfully!")

## 3. Use Case 2: Portfolio Analysis Workflow {#portfolio-analysis}

Now let's build a portfolio analysis workflow that creates detailed analytics and investment committee presentations.

### 3.1 Portfolio Analytics Excel {#portfolio-excel}

Create a comprehensive portfolio analysis workbook with performance metrics, risk analysis, and rebalancing recommendations.

**⏱️ Expected time:** 1-2 minutes

In [ ]:
print("Creating portfolio analysis Excel workbook...")
print("\n⏱️ Generation time: 1-2 minutes\n")

# Prepare portfolio data for the prompt
top_holdings = portfolio_df.nlargest(5, 'market_value')
sector_allocation = portfolio_data['sector_allocation']

portfolio_excel_prompt = f"""
Create a portfolio analysis Excel workbook with multiple sheets:

Sheet 1 - "Holdings Overview":
Create a detailed holdings table with these positions:
{portfolio_df[['ticker', 'name', 'shares', 'current_price', 'market_value', 'unrealized_gain', 'allocation_percent']].to_string()}

Add:
- Total portfolio value: ${portfolio_data['total_value']:,.2f}
- Total unrealized gain: ${portfolio_df['unrealized_gain'].sum():,.2f}
- Conditional formatting: green for gains, red for losses

Sheet 2 - "Performance Analysis":
Create performance metrics:
- Total Return: {portfolio_data['performance_metrics']['total_return_percent']:.1f}%
- YTD Return: {portfolio_data['performance_metrics']['year_to_date_return']:.1f}%
- Annualized Return: {portfolio_data['performance_metrics']['annualized_return']:.1f}%
- Sharpe Ratio: {portfolio_data['performance_metrics']['sharpe_ratio']:.2f}
- Beta: {portfolio_data['performance_metrics']['beta']:.2f}
- Standard Deviation: {portfolio_data['performance_metrics']['standard_deviation']:.1f}%

Add a bar chart showing top 5 performers by unrealized gain.

Sheet 3 - "Sector Allocation":
Show sector breakdown:
{json.dumps(sector_allocation, indent=2)}

Include:
- Pie chart of sector allocation
- Target vs Actual allocation table
- Rebalancing recommendations

Sheet 4 - "Risk Metrics":
Create a risk dashboard with:
- Portfolio Beta: 0.85
- Value at Risk (95%): $62,500 (5% of portfolio)
- Maximum Drawdown (historical): -12.3%
- Correlation matrix for top 5 holdings
- Risk-adjusted returns chart

Apply professional formatting throughout.
"""

# Create portfolio analysis Excel
portfolio_response, portfolio_results = create_skills_message(
    client,
    portfolio_excel_prompt,
    [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}],
    prefix="portfolio_analysis_"
)

print("\n" + "="*60)
print_download_summary(portfolio_results)
print("\n✅ Portfolio analysis Excel created successfully!")

### 3.2 Investment Committee Presentation {#investment-deck}

Generate an investment committee presentation with portfolio performance, risk analysis, and recommendations.

**⏱️ Expected time:** 1-2 minutes

In [ ]:
print("Creating investment committee presentation...")
print("\n⏱️ Generation time: 1-2 minutes\n")

investment_deck_prompt = f"""
Create a 5-slide investment committee presentation:

Slide 1 - Title:
- Title: "Portfolio Review - Q4 2024"
- Subtitle: "{portfolio_data['portfolio_name']}"
- Date: January 2025
- Portfolio Value: ${portfolio_data['total_value']:,.0f}

Slide 2 - Portfolio Overview:
- Title: "Portfolio Performance Summary"
- Two-column layout:
  
  Left - Key Metrics:
  • Total Value: ${portfolio_data['total_value']:,.0f}
  • YTD Return: +{portfolio_data['performance_metrics']['year_to_date_return']:.1f}%
  • Total Return: ${portfolio_data['performance_metrics']['total_return']:,.0f}
  • Sharpe Ratio: {portfolio_data['performance_metrics']['sharpe_ratio']:.2f}
  
  Right - Bar chart of top 5 holdings by value:
  {', '.join([f"{h['ticker']}: ${h['market_value']:,.0f}" for h in top_holdings.to_dict('records')])}

Slide 3 - Sector Allocation:
- Title: "Asset Allocation & Diversification"
- Pie chart showing:
  Technology: {sector_allocation['Technology']:.1f}%
  Financials: {sector_allocation['Financials']:.1f}%
  Healthcare: {sector_allocation['Healthcare']:.1f}%
  Consumer: {sector_allocation['Consumer Discretionary'] + sector_allocation['Consumer Staples']:.1f}%
  Fixed Income: {sector_allocation['Bonds']:.1f}%
  Cash: {sector_allocation['Cash']:.1f}%

Slide 4 - Risk Analysis:
- Title: "Risk Metrics & Analysis"
- Content:
  Risk Indicators:
  • Portfolio Beta: {portfolio_data['performance_metrics']['beta']:.2f} (lower market risk)
  • Standard Deviation: {portfolio_data['performance_metrics']['standard_deviation']:.1f}%
  • Maximum Drawdown: -12.3%
  • Value at Risk (95%): $62,500
  
  Risk-Adjusted Performance:
  • Sharpe Ratio: {portfolio_data['performance_metrics']['sharpe_ratio']:.2f} (excellent)
  • Alpha Generation: +2.3% vs benchmark

Slide 5 - Recommendations:
- Title: "Strategic Recommendations"
- Bullet points:
  ✓ Maintain current allocation - well diversified
  ✓ Consider profit-taking in Technology (20% → 18%)
  ✓ Increase Healthcare allocation (8.7% → 10%)
  ✓ Monitor bond duration given rate environment
  ✓ Rebalance quarterly to maintain targets

Use professional investment presentation design.
"""

# Create investment committee deck
investment_response, investment_results = create_skills_message(
    client,
    investment_deck_prompt,
    [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}],
    prefix="investment_committee_"
)

print("\n" + "="*60)
print_download_summary(investment_results)
print("\n✅ Investment committee presentation created successfully!")

## 4. Use Case 3: Automated Reporting Pipeline {#reporting-pipeline}

Let's demonstrate how to chain multiple skills together to create an automated reporting workflow that goes from data → Excel → PowerPoint → PDF.

This showcases:
- Multi-step document generation
- Cross-format data flow
- Token usage optimization
- Production automation patterns

**⏱️ Total expected time:** 2-3 minutes for the complete pipeline

In [ ]:
print("🔄 Starting Automated Reporting Pipeline")
print("="*60)
print("This will create a complete reporting suite:")
print("1. Excel analysis → 2. PowerPoint summary → 3. PDF documentation")
print("\n⏱️ Total pipeline time: 2-3 minutes\n")

# Track token usage across the pipeline
pipeline_tokens = {'input': 0, 'output': 0}

# Step 1: Create Excel Analysis
print("Step 1/3: Creating Excel analysis with quarterly metrics...")

excel_pipeline_prompt = f"""
Create a quarterly business metrics Excel file:

Sheet 1 - "Quarterly KPIs":
Create a table with these quarterly metrics for 2024:
{json.dumps([{k: v for k, v in q.items() if k in ['quarter', 'revenue', 'gross_margin', 'customer_count', 'churn_rate']} 
            for q in quarterly_metrics['quarters']], indent=2)}

Add:
- Quarter-over-quarter growth calculations
- Average and total rows
- Conditional formatting for trends
- Line chart showing revenue trend
- Column chart showing customer count

Sheet 2 - "YoY Comparison":
Compare Q4 2024 vs Q4 2023 for all metrics.
Calculate percentage changes and highlight improvements.

Professional formatting with headers and borders.
"""

excel_response, excel_results = create_skills_message(
    client,
    excel_pipeline_prompt,
    [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}],
    prefix="pipeline_1_metrics_",
    show_token_usage=False
)

pipeline_tokens['input'] += excel_response.usage.input_tokens
pipeline_tokens['output'] += excel_response.usage.output_tokens
print(f"✓ Excel created - Tokens: {excel_response.usage.input_tokens} in, {excel_response.usage.output_tokens} out")

# Step 2: Create PowerPoint Summary
print("\nStep 2/3: Creating PowerPoint summary from metrics...")

ppt_pipeline_prompt = f"""
Create a 3-slide quarterly metrics summary presentation:

Slide 1:
- Title: "Q4 2024 Metrics Summary"
- Subtitle: "Automated Reporting Pipeline Demo"

Slide 2:
- Title: "Key Performance Indicators"
- Show Q4 2024 metrics:
  • Revenue: $3.2M (+15% QoQ)
  • Customers: 850 (+8.9% QoQ)
  • Gross Margin: 72%
  • Churn Rate: 2.8% (improved from 3.5%)
- Add a simple bar chart comparing Q3 vs Q4 revenue

Slide 3:
- Title: "Quarterly Trend Analysis"
- Line chart showing revenue growth Q1-Q4
- Key insight bullets:
  • Consistent QoQ growth
  • Customer acquisition accelerating
  • Churn reduction successful

Clean, data-focused design.
"""

ppt_response, ppt_results = create_skills_message(
    client,
    ppt_pipeline_prompt,
    [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}],
    prefix="pipeline_2_summary_",
    show_token_usage=False
)

pipeline_tokens['input'] += ppt_response.usage.input_tokens
pipeline_tokens['output'] += ppt_response.usage.output_tokens
print(f"✓ PowerPoint created - Tokens: {ppt_response.usage.input_tokens} in, {ppt_response.usage.output_tokens} out")

# Step 3: Create PDF Documentation
print("\nStep 3/3: Creating PDF documentation...")

pdf_pipeline_prompt = """
Create a PDF document summarizing the quarterly reporting pipeline:

AUTOMATED REPORTING PIPELINE
Q4 2024 Results Documentation

EXECUTIVE SUMMARY
This document summarizes the Q4 2024 business metrics generated through
our automated reporting pipeline.

KEY METRICS
- Revenue: $3.2M (15% QoQ growth)
- Customer Base: 850 active customers
- Gross Margin: 72%
- Churn Rate: 2.8% (improved from 3.5%)

PIPELINE COMPONENTS
1. Data Processing: Quarterly metrics analyzed in Excel
2. Visualization: Key insights presented in PowerPoint
3. Documentation: Formal report generated in PDF

AUTOMATION BENEFITS
• Reduced reporting time by 90%
• Consistent format and quality
• Eliminated manual errors
• Scalable to multiple reports

NEXT STEPS
- Expand pipeline to include predictive analytics
- Add automated email distribution
- Implement real-time data feeds

Generated: January 2025
Pipeline Version: 1.0

Format as a professional technical document.
"""

pdf_response, pdf_results = create_skills_message(
    client,
    pdf_pipeline_prompt,
    [{"type": "anthropic", "skill_id": "pdf", "version": "latest"}],
    prefix="pipeline_3_documentation_",
    show_token_usage=False
)

pipeline_tokens['input'] += pdf_response.usage.input_tokens
pipeline_tokens['output'] += pdf_response.usage.output_tokens
print(f"✓ PDF created - Tokens: {pdf_response.usage.input_tokens} in, {pdf_response.usage.output_tokens} out")

# Pipeline Summary
print("\n" + "="*60)
print("🎯 PIPELINE COMPLETE!")
print("="*60)

print("\n📊 Pipeline Token Usage Summary:")
print(f"   Total Input Tokens: {pipeline_tokens['input']:,}")
print(f"   Total Output Tokens: {pipeline_tokens['output']:,}")
print(f"   Total Tokens: {pipeline_tokens['input'] + pipeline_tokens['output']:,}")
print(f"   Average per document: {(pipeline_tokens['input'] + pipeline_tokens['output']) // 3:,}")

print("\n📁 Generated Files:")
all_results = excel_results + ppt_results + pdf_results
for i, result in enumerate(all_results, 1):
    if result['success']:
        print(f"   {i}. {os.path.basename(result['output_path'])} ({result['size'] / 1024:.1f} KB)")

print("\n✅ Automated reporting pipeline executed successfully!")
print("   All three documents created and linked in workflow.")

## 5. Best Practices & Production Tips {#best-practices}

Based on our examples, here are key recommendations for using Skills in production financial applications:

### Token Usage Optimization

1. **Batch Operations**: Combine related requests in a single prompt
2. **Use Structured Data**: JSON/CSV format reduces tokens vs prose
3. **Skill Reuse**: The skill loads once per conversation
4. **Specific Instructions**: Clear requirements reduce iterations

### Error Handling Patterns

In [ ]:
def robust_skills_request(client, prompt, skills, max_retries=3, prefix=""):
    """
    Production-ready Skills request with error handling and retries.
    """
    from time import sleep
    from anthropic import RateLimitError, APIError
    
    for attempt in range(max_retries):
        try:
            response = client.beta.messages.create(
                model=MODEL,
                max_tokens=4096,
                container={"skills": skills},
                tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
                messages=[{"role": "user", "content": prompt}],
                betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"]
            )
            
            # Extract and download files
            file_ids = extract_file_ids(response)
            if not file_ids:
                print(f"Warning: No files found in response (attempt {attempt + 1})")
                if attempt < max_retries - 1:
                    continue
            
            # Download files
            results = download_all_files(
                client,
                response,
                output_dir=str(OUTPUT_DIR),
                prefix=prefix
            )
            
            # Verify at least one successful download
            if any(r['success'] for r in results):
                return response, results
            else:
                print(f"Warning: No successful downloads (attempt {attempt + 1})")
                
        except RateLimitError as e:
            wait_time = min(2 ** attempt * 2, 30)  # Max 30 seconds
            print(f"Rate limited. Waiting {wait_time}s...")
            sleep(wait_time)
            
        except APIError as e:
            print(f"API Error (attempt {attempt + 1}): {e}")
            if attempt == max_retries - 1:
                raise
            sleep(2)
            
        except Exception as e:
            print(f"Unexpected error (attempt {attempt + 1}): {e}")
            if attempt == max_retries - 1:
                raise
    
    raise Exception(f"Failed after {max_retries} attempts")

print("✓ Production error handling pattern defined")

### Performance Guidelines

**Document Complexity vs Generation Time:**

| Complexity | Excel | PowerPoint | PDF | Time |
|------------|-------|------------|-----|------|
| Simple (1-2 sheets/slides) | ✓ | ✓ | ✓ | 40-60s |
| Medium (3-4 sheets/slides) | ✓ | ✓ | ✓ | 1-2 min |
| Complex (5+ sheets/slides) | ✓ | ⚠️ | ✓ | 2-3 min |
| Very Complex (10+ elements) | ⚠️ | ❌ | ⚠️ | 3-5+ min |

**Recommendations:**
- Keep individual documents focused and under 5 sheets/slides
- Break complex reports into multiple smaller documents
- Use pipelines to chain simpler operations
- Monitor generation times and adjust complexity

### Production Deployment Checklist

- [ ] Implement retry logic with exponential backoff
- [ ] Add file validation after download
- [ ] Set up monitoring for token usage
- [ ] Create fallback templates for critical reports
- [ ] Implement caching for frequently used data
- [ ] Add logging for audit trails
- [ ] Set up alerts for failed generations
- [ ] Test with production data volumes
- [ ] Document expected generation times
- [ ] Plan for API rate limits

### Cost Optimization Strategies

1. **Template Reuse**: Create base templates and modify with data
2. **Batch Processing**: Group similar reports in single sessions
3. **Incremental Updates**: Only regenerate changed sections
4. **Off-Peak Processing**: Schedule large jobs during lower-usage periods
5. **Data Preprocessing**: Format data efficiently before sending

In [ ]:
# Example: Token usage tracking
class TokenTracker:
    def __init__(self):
        self.total_input = 0
        self.total_output = 0
        self.requests = 0
    
    def track(self, response):
        self.total_input += response.usage.input_tokens
        self.total_output += response.usage.output_tokens
        self.requests += 1
    
    def report(self):
        total = self.total_input + self.total_output
        avg_per_request = total / self.requests if self.requests > 0 else 0
        
        print("Token Usage Report:")
        print(f"  Total Requests: {self.requests}")
        print(f"  Input Tokens: {self.total_input:,}")
        print(f"  Output Tokens: {self.total_output:,}")
        print(f"  Total Tokens: {total:,}")
        print(f"  Average per Request: {avg_per_request:,.0f}")
        
        # Rough cost estimate (example rates)
        input_cost = self.total_input * 0.003 / 1000  # $3 per 1M tokens
        output_cost = self.total_output * 0.015 / 1000  # $15 per 1M tokens
        total_cost = input_cost + output_cost
        
        print(f"\nEstimated Cost:")
        print(f"  Input: ${input_cost:.2f}")
        print(f"  Output: ${output_cost:.2f}")
        print(f"  Total: ${total_cost:.2f}")
        print(f"  Per Request: ${total_cost/self.requests:.3f}")

# Demo with our pipeline data
tracker = TokenTracker()
tracker.total_input = pipeline_tokens['input']
tracker.total_output = pipeline_tokens['output']
tracker.requests = 3

print("Pipeline Token Analysis:")
print("="*40)
tracker.report()

## Summary & Next Steps

### What We've Accomplished

In this notebook, you've learned how to:

✅ **Financial Dashboard Creation**
- Built multi-sheet Excel models with formulas and charts
- Generated executive PowerPoint presentations
- Created professional PDF reports

✅ **Portfolio Analysis**
- Developed portfolio analytics workbooks
- Created investment committee presentations
- Implemented risk metrics and rebalancing tools

✅ **Automated Pipelines**
- Chained multiple document formats
- Optimized token usage
- Built production-ready patterns

### Key Takeaways

1. **Skills dramatically simplify financial document creation** - What would take hours manually takes minutes
2. **Token efficiency is excellent** - Skills use ~90% fewer tokens than manual instructions
3. **Quality is professional-grade** - Documents are immediately usable in business contexts
4. **Automation is straightforward** - Pipeline patterns enable complex workflows

### Continue Your Learning

📚 **Next: [Notebook 3 - Custom Skills Development](03_skills_custom_development.ipynb)**
- Build your own specialized financial skills
- Create company-specific templates
- Implement advanced automation

### Try These Experiments

1. **Modify the financial dashboard** to include your own metrics
2. **Create a custom portfolio** with different asset classes
3. **Build a pipeline** for your specific reporting needs
4. **Experiment with complexity** to understand generation times
5. **Track token usage** across different document types

### Resources

- [Claude API Documentation](https://docs.anthropic.com/en/api/messages)
- [Skills Documentation](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/skills)
- [Files API Reference](https://docs.claude.com/en/api/files-content)